# Polar Expression Plugins with `rustimport_jupyter`

For Google colab, we install [rustimport_jupyter](https://github.com/thomasjpfan/rustimport_jupyter) and the rust toolchain:

In [1]:
import os
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install rustimport_jupyter polars==0.20.2
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] += ":/root/.cargo/bin"

In [1]:
%load_ext rustimport_jupyter

## Define the polars expression

As show in [Polar's documentation](https://pola-rs.github.io/polars/user-guide/expressions/plugins/), we can define a pig-latin expression in rust. We'll need the compiled module later, so we set `--module-path-variable=polars_pig_latin_module`:

In [4]:
%%rustimport --release --module-path-variable=polars_pig_latin_module
//: [dependencies]
//: polars = { version = "*" }
//: pyo3 = { version = "*", features = ["extension-module"] }
//: pyo3-polars = { version = "0.9", features = ["derive"] }
//: serde = { version = "*", features = ["derive"] }

use pyo3::prelude::*;
use polars::prelude::*;
use pyo3_polars::derive::polars_expr;
use std::fmt::Write;

fn pig_latin_str(value: &str, output: &mut String) {
    if let Some(first_char) = value.chars().next() {
        write!(output, "{}{}ay", &value[1..], first_char).unwrap()
    }
}

#[polars_expr(output_type=Utf8)]
fn pig_latinnify(inputs: &[Series]) -> PolarsResult<Series> {
    let ca = inputs[0].utf8()?;
    let out: Utf8Chunked = ca.apply_to_buffer(pig_latin_str);
    Ok(out.into_series())
}

    Updating crates.io index
   Compiling autocfg v1.1.0
   Compiling version_check v0.9.4
   Compiling libc v0.2.151
   Compiling proc-macro2 v1.0.71
   Compiling unicode-ident v1.0.12
   Compiling libm v0.2.8
   Compiling crossbeam-utils v0.8.18
   Compiling rayon-core v1.12.0
   Compiling cfg-if v1.0.0
   Compiling thiserror v1.0.52
   Compiling target-features v0.1.5
   Compiling syn v1.0.109
   Compiling once_cell v1.19.0
   Compiling heck v0.4.1
   Compiling memchr v2.6.4
   Compiling either v1.9.0
   Compiling zerocopy v0.7.32
   Compiling rustversion v1.0.14
   Compiling regex-syntax v0.8.2
   Compiling parking_lot_core v0.9.9
   Compiling ahash v0.8.6
   Compiling smartstring v1.0.1
   Compiling polars-utils v0.35.4
   Compiling num-traits v0.2.17
   Compiling aho-corasick v1.1.2
   Compiling lock_api v0.4.11
   Compiling quote v1.0.33
   Compiling getrandom v0.2.11
   Compiling syn v2.0.43
   Compiling crossbeam-epoch v0.9.17
   Compiling smallvec v1.11.2
   Compiling scopegu

We install dependencies required by Polars using with a `//:` comment syntax, which will be injected into a `Cargo.toml` file.

With `polars_pig_latin_module` defined, we can define a `language` namespace for the Polars dataframe:

In [5]:
import polars as pl

@pl.api.register_expr_namespace("language")
class Language:
    def __init__(self, expr: pl.Expr):
        self._expr = expr

    def pig_latinnify(self) -> pl.Expr:
        return self._expr.register_plugin(
            lib=polars_pig_latin_module,
            symbol="pig_latinnify",
            is_elementwise=True,
        )

With the `language` namespace defined, we can now use it with Polars:

In [6]:
df = pl.DataFrame(
    {
        "convert": ["pig", "latin", "is", "silly"],
    }
)

out = df.with_columns(
    pig_latin=pl.col("convert").language.pig_latinnify(),
)
print(out)

shape: (4, 2)
┌─────────┬───────────┐
│ convert ┆ pig_latin │
│ ---     ┆ ---       │
│ str     ┆ str       │
╞═════════╪═══════════╡
│ pig     ┆ igpay     │
│ latin   ┆ atinlay   │
│ is      ┆ siay      │
│ silly   ┆ illysay   │
└─────────┴───────────┘
